James Caldwell <br>
UVA IRA <br>
2/11/25


Code written to process and merge NSC application (accepted and not accepted students) with UVA Admission demographics. Used for scholarship award decisions. Data request 560.

In [355]:
# Clear all variables in the current environment
from IPython import get_ipython
get_ipython().magic('reset -f')

C:\Users\ywe4kw\AppData\Local\Temp\ipykernel_14172\2666348552.py:3: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -f')


In [356]:
import pandas as pd
import numpy as np

In [357]:
# nsc = pd.read_csv('003745st_601528_DETLRPT_DA_11182024112349_deniedadmission_f14f24_003745.csv',usecols=['Requester Return Field'],nrows=1000)
nsc = pd.read_csv('003745st_601528_DETLRPT_DA_11182024112349_deniedadmission_f14f24_003745.csv',nrows=30000, dtype=str)

In [358]:
#for creating unduplicated list
yesno = 0
if yesno:
    len(nsc)
    nsc_unduplicated = nsc.drop_duplicates()
    len(nsc_unduplicated)
    nsc_unduplicated.to_csv('nsc_unduplicated.csv')

In [368]:
qlik = pd.read_excel('Qlik Ugrad Admissions Data.xlsx',nrows=30000)


In [369]:
len(qlik['Student System ID'].unique())

1063

In [361]:
qlik['Test ID'].unique()

array(['SAT I', 'SAT II', 'ACT', 'TOEFL', 'AP', 'IB', '-'], dtype=object)

In [362]:
qlik['Test Component'].unique()

array(['ES', 'M1', 'MATH', 'MC', 'VERB', 'WR', '1600', '2400', 'COMP',
       'ENGL', 'EW', 'READ', 'SCIRE', 'COMPI', 'LISCI', 'MTHS', 'READI',
       'SP', 'SPKI', 'SPNA2', 'UH', 'WRITI', 'MB', 'LR', 'MAB', 'WH',
       'MBC', 'MSS', 'BY', 'CH', 'M2', '-', 'SPNB', 'ENGA1', 'HIST', 'LT',
       'PH', 'EB', 'FR', 'CHEMY', 'SL', 'KL', 'CL', 'FL', 'GM', 'ECON',
       'FRNB', 'BIOLG', 'GL', 'MATHM', 'JL'], dtype=object)

In [ ]:
## Filter to reduce the number of tests per student

# Filter out Tests for only SAT I, ACT, or blanks
qlik = qlik[qlik['Test ID'].isin(['SAT I', 'ACT','-'])]
# Filter out SAT/ACT components to only 
    # SAT = comp, read/write, or math
    # ACT = comp, math, or english
qlik = qlik[qlik['Test Component'].isin(['COMP', 'READ', 'ENGL', 'MATH','-'])]
    # This doesn't reduce the number of unique students, just the number of rows per student

In [371]:
len(qlik['Student System ID'].unique())

1060

The following code converts long to wide format for some of variables (Subject Area Interest Desc and Test Scores).
For example, many of the students had 2 subject Area Interest Desc. There would be 2 rows for the student with all the same data except the 2 intererests. This code adds a second column for the interest name and condenses the two record rows down to one.

In [365]:
## 1: Subject Area Interest Desc
# List of columns to exclude from grouping
exclude_cols = ['Subject Area Interest Desc']
# Determine the columns to group by (all except the excluded ones)
groupby_cols = [col for col in qlik.columns if col not in exclude_cols]
# Assign Program_Number based on these grouped columns
qlik['Program_Number_counts'] = qlik.groupby(groupby_cols).cumcount() + 1

# Pivot to wide format
qlik_wide = qlik.pivot(index=groupby_cols, columns='Program_Number_counts', values='Subject Area Interest Desc')

# Rename columns dynamically
qlik_wide.columns = [f'Subject Area Interest Desc {col}' for col in qlik_wide.columns]

# Reset index to bring all other data back as columns
qlik_wide = qlik_wide.reset_index()


In [366]:
## 2: Test Scores
# Ensure 'Score' is numeric
qlik_wide['Score'] = pd.to_numeric(qlik_wide['Score'], errors='coerce')  # Converts non-numeric to NaN

# # Combine Test ID and Test Component into a new column 'Test_Combination'
qlik_wide['Test_Combination'] = qlik_wide['Test ID'] + ' ' + qlik_wide['Test Component']

# Use pivot_table to handle duplicates
qlik_test_scores = qlik_wide.pivot_table(
    index='Student System ID', 
    columns='Test_Combination', 
    values='Score', 
    aggfunc='mean',  # or 'first', 'sum', etc.
    dropna = False
)

# Reset index to make Student System ID a column again
# This is now a table (df) with one row per student, and their test scores as columns
qlik_test_scores.reset_index(inplace=True)

## Drop duplicate rows of a student that were due to multiple test scores
qlik_wide.drop('Test_Combination', axis=1,inplace=True)
exclude_cols = ['Test ID', 'Test Component','Score']
columns_to_check = [col for col in qlik_wide.columns if col not in exclude_cols]
# Drop duplicates, keeping the first occurrence
qlik_wide_no_duplicates = qlik_wide.drop_duplicates(subset=columns_to_check, keep='first')
# Optionally, reset the index if needed
qlik_wide_no_duplicates.reset_index(drop=True, inplace=True)
qlike_wide_no_duplicates2=qlik_wide_no_duplicates.drop(['Test ID', 'Test Component','Score'], axis=1)
qlik_final= pd.merge(qlike_wide_no_duplicates2, qlik_test_scores, on='Student System ID', how='left')

Remove duplicate rows due to multiple class ranks, Y and N being listed as Offered in Waitlist Flag or Rodman, Echols, Jefferson, or College Science scholar flags

In [367]:
qlik_final.sort_values(['Student System ID','Class Rank','Offered Waitlist Flag'], inplace=True)
exclude_cols = ['Class Rank','Percentile Rank','Offered Waitlist Flag']
columns_to_check = [col for col in qlik_final.columns if col not in exclude_cols and 'scholar' not in col.lower()]
qlik_final=qlik_final.drop_duplicates(subset=columns_to_check, keep='first')